In [9]:
import numpy as np
import pandas as pd
import pycountry
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
sns.set_style("darkgrid")
pd.set_option('display.float_format', lambda x:  '{:,.2f}'.format(x) if abs(x) < 10 else '{:,.0f}'.format(x)) #FORMAT PANDAS

In [10]:
df = pd.read_excel(r"C:\Users\boris.mikanikrezai\OneDrive - Euromoney Institutional Investor PLC\Desktop\TIN CONSUMPTION MODEL\Database Tin WBMS formatted.xlsx",sheet_name="Tin monthly")

df.set_index("Country",drop=True,inplace=True)
df = df[ (df["Product"]=="Ref Sn") & (df["Type"]=="Consumption")]
df.drop(["Product","Type"],axis="columns",inplace=True)
df = df.transpose()
df.index=pd.DatetimeIndex(df.index)
df_a = df.resample("A").sum()
df_a= df_a.transpose()
df_a.sort_values(["2019-12-31"],ascending=False,inplace=True)

In [11]:
df_a.reset_index(inplace=True)
df_oui = df_a.melt(id_vars=["Country"],var_name="Date",value_name="Value")

In [12]:
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3

In [14]:
df_oui["iso_alpha"] = df_oui["Country"].apply(lambda x: do_fuzzy_search(x))

In [15]:
df_oui = df_oui.rename(columns={"Value": "Consumption (tonnes)"})

In [16]:
df_oui["Date"]=df_oui["Date"].apply(lambda x: x.strftime('%Y'))

In [17]:
df_oui.set_index("Country",inplace=True)
df_oui.drop(["World Total","Asia Total","Europe Total","European Union Total","America Total","NAFTA Total"],axis=0,inplace=True)
df_oui.reset_index(inplace=True)

In [27]:
fig = px.scatter_geo(df_oui, locations="iso_alpha",
                     hover_name="Country", size="Consumption (tonnes)",
                     
                     hover_data={'iso_alpha':False, # remove species from hover data
                             'Consumption (tonnes)':':,.2f', # customize hover for column of y attribute
                                },
                     
                     color="Consumption (tonnes)",
                     color_continuous_scale=px.colors.diverging.RdBu[::-1],
                     #animation_frame="Date",
                     projection="natural earth",
                     title="Global refined tin consumption",width=1050, height=800
                     # projection="orthographic"
                     )
                     
fig.show()

In [ ]:
import plotly.graph_objects as go # or plotly.express as px
fig = fig # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter